# Assignment 1 Comp 521 Fall 2017

Assigned: 5 September 2017  
Due: 19 September 2017

Answer each question below by filling in code at the point indicated. Do not modify any of the other cells. 

You __must__ run the cells in order. It looks like you can come back later after restarting and pick up where you left off but that is not the case. When restarting you always have to start at the top of the notebook and run each cell in turn. 

In [44]:
# fill in the information below
Author = 'Zhilan Zhou'
Collaborators = []

In [45]:
# checker setup do NOT change this
import comp521
check, report = comp521.start('A1')

In [46]:
# setup, no need to change this
from Dee import *
from DeeDatabase import Database
from CourseEnrollmentDee import DeeDB
import sqlite3

# I'll open a DB in memory
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

## Question 1
Use SQL to create tables for the following database schema modeling a library.

STUDENT __sid__(INTEGER) sname(TEXT)  
BOOK __bid__(INTEGER) bname(TEXT) author(TEXT)  
BORROWS __id__(INTEGER) sid(INTEGER) bid(INTEGER) date(TEXT)  

All primary keys are bold. In the BORROWS table, __bid__ is a foreign key referencing BOOK and __sid__ is a foreign key referencing STUDENT.

__Make sure__ you use the *exact* names and data types for the tables and attributes in order to pass the automatic test.

In [47]:
# I'll clean up for you so you don't get tripped up by trying more than once. Leave this alone
cursor.execute('DROP TABLE IF EXISTS "STUDENT"')
cursor.execute('DROP TABLE IF EXISTS "BOOK"')
cursor.execute('DROP TABLE IF EXISTS "BORROWS"')

# insert your code to create the table here using cursor to create the tables.
cursor.execute("""
create table STUDENT 
    (sid integer primary key, 
     sname text)
""")

cursor.execute("""
create table BOOK
    (bid integer primary key, 
     bname text, 
     author text)
""")

cursor.execute("""
create table BORROWS 
    (id integer primary key, 
     sid integer, 
     bid integer, 
     date text,
     foreign key (sid) references STUDENT
     foreign key (bid) references BOOK)
""")

# leave this testing code alone
c1 = cursor.execute("SELECT name from sqlite_master WHERE type='table' order by name asc").fetchall()
check('Q1a', c1, points=10)

Q1a appears correct


## Question 2
Insert the following data into the tables you created above.

<table>
<caption>STUDENT</caption>
<tr><th>sid</th><th>sname</th></tr>
<tr><td>1</td><td>Alice</td></tr>
<tr><td>2</td><td>Bob</td></tr>
<tr><td>3</td><td>Cindy</td></tr>
</table>

<table>
<caption>BOOK</caption>
<tr><th>bid</th><th>bname</th><th>author</th></tr>
<tr><td>1</td><td>Database Management Systems</td><td>Ramakrishnan</td></tr>
<tr><td>2</td><td>Lord of the Rings</td><td>Tolkien</td></tr>
</table>

<table>
<caption>BORROWS</caption>
<tr><th>id</th><th>sid</th><th>bid</th><th>date</th></tr>
<tr><td>1</td><td>2</td><td>1</td><td>8/3/2017</td></tr>
<tr><td>2</td><td>3</td><td>2</td><td>9/1/2017</td></tr>
</table>


In [48]:
# write your code here
students = [(1, 'Alice'),
            (2, 'Bob'),
            (3, 'Cindy')]
for student in students:
    cursor.execute("insert into STUDENT values(?, ?)", student)
    
books = [(1, 'Database Management Systems', 'Ramakrishnan'),
         (2, 'Lord of the Rings', 'Tolkien')]
for book in books:
    cursor.execute("insert into BOOK values(?, ?, ?)", book)

borrows = [(1, 2, 1, '8/3/2017'),
         (2, 3, 2, '9/1/2017')]
for borrow in borrows:
    cursor.execute("insert into BORROWS values(?, ?, ?, ?)", borrow)
    
# testing code
c2a = cursor.execute('select * from STUDENT').fetchall()
check('Q2a', c2a, points=5)
c2b = cursor.execute('select * from BOOK').fetchall()
check('Q2b', c2b, points=5)
c2c = cursor.execute('select * from BORROWS').fetchall()
check('Q2c', c2c, points=5)

Q2a appears correct
Q2b appears correct
Q2c appears correct


## Question 3
Delete the record with id equal 1 from the BORROWS table.

In [49]:
# write your code here
cursor.execute("""
delete from BORROWS
    where id=1
""")
    
# don't touch this
c3 = cursor.execute('''select * from BORROWS''').fetchall()
check('Q3', c3, points=10)

Q3 appears correct


## Setup for questions 4-11
Consider the following schema for a database modeling courses at UNC.
```
    DEPARTMENT (did, name)  
    COURSE (cid, did, name, num, creditHours)  
    STUDENT (sid, fname, lname, did)  
    ENROLLED_IN (eid, sid, cid)  
```
    
In the COURSE table, did is a foreign key referencing DEPARTMENT(did). In the ENROLLED_IN
table, sid is a foreign key referencing STUDENT(sid), and cid is a foreign key referencing
COURSE(cid). In STUDENT, did is a foreign key referencing DEPARTMENT(did).

### Questions 4-7

Use [Dee](http://www.quicksort.co.uk/DeeDoc.html) to write relational algebra queries for the following. In particular, you may find the
following Dee operators most useful: Projection, Restriction (corresponds to select in
relational algebra), Join, Intersection, Difference, Union.

Your, already populated, Dee database is named DeeDB.


## Question 4
Find the names of all courses below the 500 level. You can get the Dee relation for COURSE with `DeeDB.COURSE`.

In [50]:
# write code here used DeeDB and Dee methods.
c4 = DeeDB.COURSE.where(lambda t: t.num < 500).project(['name'])

# checking code
check('Q4', c4, points=5)

Q4 appears correct


## Question 5
Find the first and last names of all students enrolled in "Files and Databases".

In [51]:
# Assign your result to c5
c5 = (DeeDB.STUDENT.remove(['did']) & DeeDB.ENROLLED_IN & DeeDB.COURSE.where
 (lambda t: t.name == "Files and Databases")).project(['fname','lname'])

# checking
check('Q5', c5, points=5)

Q5 appears correct


## Question 6
Find the first and last names of all students in the ANTH department.

In [52]:
# assign to c6
c6 = (DeeDB.STUDENT & DeeDB.DEPARTMENT.where
 (lambda t: t.name == "ANTH")).project(['fname','lname'])

# checking
check('Q6', c6, points=5)

Q6 appears correct


## Question 7
List the first and last names of all students enrolled in no courses.

In [53]:
# assign to c7
c7 = ((DeeDB.STUDENT.project(['fname','lname'])) - 
((DeeDB.STUDENT & DeeDB.ENROLLED_IN & DeeDB.COURSE).project(['fname','lname'])))

# checking
check('Q7', c7, points=5)

Q7 appears correct


### Questions 8-11

Now use the sqlite3 database `courseEnrollment.db`, which has the same schema as above to write SQL queries for the following. I will open it for you below. __You must execute this next cell before the others!__

In [54]:
econn = sqlite3.connect('CourseEnrollment.db')
ecursor = econn.cursor()

# use ecursor to refer to this database below.

## Question 8
Find the names of all 4 credit hour courses.

In [55]:
# your code here
c8 = ecursor.execute("""
select C.name 
    from COURSE C
    where C.creditHours=4""").fetchall()

check('Q8', c8, points=10, sort=True)

Q8 appears correct


## Question 9
Find the first and last names of all students enrolled in COMP courses.

In [56]:
# your code here
c9 = ecursor.execute("""
select distinct S.fname, S.lname
    from COURSE C, ENROLLED_IN E, STUDENT S, DEPARTMENT D
    where S.sid=E.sid and E.cid=C.cid and C.did=D.did and D.name='COMP'""").fetchall()

check('Q9', c9, points=10, sort=True)

Q9 appears correct


## Question 10
Find the first and last names of all students enrolled in courses of at least 500 level.

In [57]:
# your code here
c10 = ecursor.execute("""
select distinct S.fname, S.lname 
    from COURSE C, ENROLLED_IN E, STUDENT S
    where S.sid=E.sid and E.cid=C.cid and C.num>500""").fetchall()

check('Q10', c10, points=10, sort=True)

Q10 appears correct


## Question 11
Find the first and last names of all students enrolled in 'Data Structures' and "Anthropology of the Body and the Subject'.

In [58]:
# your code here
c11 = ecursor.execute("""
select distinct S.fname, S.lname 
    from COURSE C1, COURSE C2, ENROLLED_IN E1, ENROLLED_IN E2, STUDENT S
    where S.sid=E1.sid and E1.cid=C1.cid and C1.name='Data Structures' and
          S.sid=E2.sid and E2.cid=C2.cid and C2.name='Anthropology of the Body and the Subject'""").fetchall()

check('Q11', c11, points=15, sort=True)

Q11 appears correct


## Done!

Now get your report and submit your assignment.

In [59]:
report(Author, Collaborators)

Q10
Q11
Q1a
Q2a
Q2b
Q2c
Q3
Q4
Q5
Q6
Q7
Q8
Q9
  Collaborators: []
Report for Zhilan Zhou
  13 of 13 correct, 100 of 100 points
